```{contents}
```
## Thread ID

In LangGraph, a **Thread ID** is the fundamental identifier that represents a **single continuous execution context** of a graph over time.
It enables **state persistence, memory continuity, resumability, concurrency isolation, and multi-user scaling**.

---

### **1. Intuition**

Without Thread IDs, every graph invocation would be **stateless** and independent.

With Thread IDs, LangGraph becomes a **long-running, stateful system**.

> **Thread ID = the identity of a conversation / workflow / agent instance**

Each Thread ID owns:

* its **state**
* its **memory**
* its **checkpoints**
* its **execution history**

---

### **2. Why Thread ID Exists**

| Problem           | Without Thread ID | With Thread ID |
| ----------------- | ----------------- | -------------- |
| State continuity  | Lost              | Preserved      |
| Multi-turn agents | Impossible        | Native         |
| Parallel users    | Collisions        | Isolated       |
| Crash recovery    | No                | Yes            |
| Human-in-loop     | Hard              | Natural        |

---

### **3. Thread ID Lifecycle**

```
Client Request
     ↓
Thread ID Created / Retrieved
     ↓
Graph Executes on Thread
     ↓
State Checkpointed
     ↓
Execution Paused / Continued Later
```

The same Thread ID can live for **minutes, days, or months**.

---

### **4. Formal Definition**

> A Thread ID is a **unique key** that maps to a **persistent state machine instance** of a compiled LangGraph.

---

### **5. How Thread ID Works Internally**

For each Thread ID, LangGraph maintains:

| Component        | Description                |
| ---------------- | -------------------------- |
| State Store      | Current shared state       |
| Checkpoint Store | Historical snapshots       |
| Execution Cursor | Where the graph paused     |
| Message History  | Conversation memory        |
| Metadata         | User, timestamps, policies |

---

### **6. Using Thread ID in Code**

```python
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

config = {
    "configurable": {
        "thread_id": "user_42_chat"
    }
}

result1 = graph.invoke({"input": "Hello"}, config=config)
result2 = graph.invoke({"input": "Continue"}, config=config)
```

Both calls operate on **the same evolving state**.

---

### **7. Thread ID vs Stateless Invocation**

| Feature                  | Stateless | With Thread ID |
| ------------------------ | --------- | -------------- |
| Memory                   | ❌         | ✅              |
| Multi-turn dialogue      | ❌         | ✅              |
| Recovery after crash     | ❌         | ✅              |
| Long workflows           | ❌         | ✅              |
| Human approval workflows | ❌         | ✅              |

---

### **8. Thread ID and Checkpointing**

Every step of execution is checkpointed:

```python
graph = builder.compile(checkpointer=memory)
```

This allows:

* resume after crash
* replay from any step
* time travel debugging

All scoped by **Thread ID**.

---

### **9. Production Use Cases**

| Use Case          | How Thread ID Is Used           |
| ----------------- | ------------------------------- |
| Chatbots          | One Thread per conversation     |
| Agents            | One Thread per agent instance   |
| Workflows         | One Thread per business process |
| Human review      | One Thread per case             |
| Multi-tenant apps | One Thread per user/session     |

---

### **10. Thread ID in Distributed Systems**

In production:

```
User → API → Thread ID → LangGraph Cluster
```

Thread ID becomes the **routing key** for:

* load balancing
* state retrieval
* execution continuation
* fault recovery

---

### **11. Common Pitfalls**

| Mistake                    | Consequence   |
| -------------------------- | ------------- |
| Reusing Thread IDs         | Data leakage  |
| Not persisting checkpoints | Lost memory   |
| Stateless configs          | No continuity |
| Unbounded thread growth    | Memory leak   |

---

### **12. Mental Model**

Think of a Thread ID as:

> **A database primary key for a living AI workflow**

Each thread is a **mini operating system process** for your agent.

In [38]:
from typing import TypedDict, List, Annotated
from langgraph.graph import add_messages

class State(TypedDict):
    messages: Annotated[List[str], add_messages]


In [39]:
def chat_node(state: State):
    user_msg = state["messages"][-1]
    return {
        "messages": [f"Assistant: I received -> {user_msg}"]
    }


In [40]:
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver

builder = StateGraph(State)
builder.add_node("chat", chat_node)
builder.set_entry_point("chat")

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [41]:
config = {
    "configurable": {
        "thread_id": "conversation_001"
    }
}

# First message
out1 = graph.invoke({"messages": ["Hello"]}, config=config)
# print(out1)

# Second message, same Thread ID
out2 = graph.invoke({"messages": ["How are you?"]}, config=config)
print(out2)


{'messages': [HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}, id='3c1fed5c-8896-4e4a-b340-08af65ad0eef'), HumanMessage(content="Assistant: I received -> content='Hello' additional_kwargs={} response_metadata={} id='3c1fed5c-8896-4e4a-b340-08af65ad0eef'", additional_kwargs={}, response_metadata={}, id='a60396dd-d046-4512-ac8f-3acd8b91b3d1'), HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={}, id='66bb0ec9-a5c6-40c8-9ff9-11e2007b02d8'), HumanMessage(content="Assistant: I received -> content='How are you?' additional_kwargs={} response_metadata={} id='66bb0ec9-a5c6-40c8-9ff9-11e2007b02d8'", additional_kwargs={}, response_metadata={}, id='fe38a5c3-9741-48c8-bd09-1ee0c4ea28f7')]}


In [42]:
memory_value = memory.get(config)

In [44]:
memory_value["channel_values"]['messages']

[HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}, id='3c1fed5c-8896-4e4a-b340-08af65ad0eef'),
 HumanMessage(content="Assistant: I received -> content='Hello' additional_kwargs={} response_metadata={} id='3c1fed5c-8896-4e4a-b340-08af65ad0eef'", additional_kwargs={}, response_metadata={}, id='a60396dd-d046-4512-ac8f-3acd8b91b3d1'),
 HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={}, id='66bb0ec9-a5c6-40c8-9ff9-11e2007b02d8'),
 HumanMessage(content="Assistant: I received -> content='How are you?' additional_kwargs={} response_metadata={} id='66bb0ec9-a5c6-40c8-9ff9-11e2007b02d8'", additional_kwargs={}, response_metadata={}, id='fe38a5c3-9741-48c8-bd09-1ee0c4ea28f7')]